In [ ]:
!pip install langchain-community chromadb

In [ ]:
!pip install langchain pypdf

In [ ]:
!pip install sentence-transformers

In [3]:
import os
from getpass import getpass

HF_TOKEN = getpass("Enter HF Token:")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN

Enter HF Token:··········


In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.llms import HuggingFaceHub

## Load and Chunk the data

Using the Langchain loader, you can load the document. For the chunking we will use Recursive character text splitter

In [6]:
documents = PyPDFLoader("Tarun_CV.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

## Index your document

In [8]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_TOKEN,model_name = "thenlper/gte-large"
)

In [11]:
db = Chroma.from_documents(texts, embeddings)

## Retriever - Normal Vector search retriever

In [12]:
retriever = db.as_retriever(
    search_kwargs={"k": 4}
)

Now the above retriever will retrieve 4 documents based on your query. But which among them is the most relevant, in order to rank that we use reranking.

## Time to Re-rank the documents

In [13]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

### In our case we are using Cross-Encoders, that is open source and we shall use HuggingFace

In [16]:
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")

compressor = CrossEncoderReranker(model=model, top_n=3)
re_rank_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authenticatio

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

## Define LLM and its relevant prompt template

Here we will use HuggingFace LLM that is open source and free to inference

In [27]:
model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
                       model_kwargs = {"temperature":0.5,
                                       "max_new_tokens":512,
                                       "max_length":64,
                                       "return_full_text":False})

In [18]:
query = "What is Tarun's contribution at AI Planet?"

In [19]:
template = """
<|system|>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [28]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

## LCEL - Langchain Expression Language

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [30]:
output_parser = StrOutputParser()

In [31]:
chain = (
    {"context": re_rank_retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [32]:
response = chain.invoke(query)

In [33]:
print(response)

Tarun's contribution at AI Planet is not explicitly mentioned in the given context. However, it is mentioned that AI Planet currently has 16K+ followers on LinkedIn and 9K+ subscribers on YouTube, and Tarun is the lead curriculum contributor to the LLM Bootcamp, where he reached out to 11 speakers and led a group of 8 AI Ambassadors for the AI Changemaker program. It is possible that Tarun's contribution at AI Planet is related to these programs and initiatives, but it is not explicitly stated in the given context.
